In [2]:
# import necessary libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# BigQuery settings
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# ignore if local tmp.csv already exists
# read the time matching CSV and remove MRNs for security reasons (ignore if local tmp.csv already exists)
import pandas as pd
data_frame = pd.read_csv('jon_mapping.csv')
data_frame = data_frame.drop('MRN', axis = 1) 
data_frame.to_csv('tmp.csv', index = False)

In [ ]:
# only for testing
# Generate a random mapping (CSV file) to test (only for testing)

import pandas as pd
data_frame = pd.read_csv('tmp.csv')
print(list(data_frame.columns)) # print the column names

num_row = data_frame.shape[0]
print(num_row) # print num_row

data_frame = data_frame.drop('JITTER', axis = 1) # remove a column

data_frame['JITTER_test']= np.random.randint(10, size=num_row) # add a column 

print(data_frame[0:10]) # print the first 10 rows

data_frame.to_csv('tmp_rnd_shift.csv', index = False) # save to CSV

In [11]:
## Create the dictionary of patients (keys are encounter ids)

# load text tole_stat.csv; this file contains different user types and total counts
import csv
with open('features/t2tpa.csv', 'r') as f:
    A_temp = csv.reader(f)
    temp = list(A_temp)

print(temp[0]) # The column names

A = np.array(temp[1:]) # arrat of t2tpa
print(A.shape)
Enc_ID = A[:,1]  


# Create the patient dictionary and extract TPA times for each encounter and put it in a dictionary

Pat_dic = {}

for id in Enc_ID:
    #print(id)
    #print(np.where(A[:,1]==id)[0])
    #print(A[np.where(A[:,1]==id)[0],2])
    t2tpa = int(A[np.where(A[:,1]==id)[0],2])
    pat_id = A[np.where(A[:,1]==id)[0],0]
    
    #print(t2tpa)
    if t2tpa < 60*10:
        Pat_dic[id] = {} #each patient has a dictionary
        Pat_dic[id]['t2tpa'] = t2tpa
        Pat_dic[id]['pat_id'] = pat_id

['jc_uid', 'pat_enc_csn_id_coded', 't2tpa']
(246, 3)


In [4]:
# Create a table of cohort AL (v1: only logs within 6 hours before tpa) with real times and Real time data into numpy

client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

query =  """
SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
  FROM `noshad.cohort_AL_user_role` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
  ORDER BY AL.rit_uid
"""
print('Query completed')
cursor.execute(query);

# into a numpy array
results = cursor.fetchall();
AL_np = np.array(results)

print(AL_np[0])

print('Real time data into numpy')



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
Query completed


KeyboardInterrupt: 

In [ ]:
# Run if I only want the treatment team information
# extract team members and size for each patient where the number of actions for each team member > AL_limit
AL_limit=3
i=0

AL_np[np.where(AL_np[:,16]==None)[0],16] = 'Not Specified'   # convert None type users to Not Specified

for id in Pat_dic.keys():
    
    # extract team members
    users , user_counts = np.unique(AL_np[np.where(AL_np[:,2]==int(id))[0],16], return_counts = True)
    Pat_dic[id]['team_size'] = np.sum(user_counts>=AL_limit)
    Pat_dic[id]['team_members_id'] = users[np.where(user_counts>=AL_limit)[0]]
    
    # report the team members of some of the cohort patients
    if i<=3:
        print(Pat_dic[id]['team_members_id'])
        #print(Pat_dic[id]['user_prev_expr'])
        #print(Pat_dic[id]['team_members_role'])
    
    i = i+1





In [ ]:
# extract team members and size for each patient where the number of actions for each team member > AL_limit
AL_limit=3
i=0

AL_np[np.where(AL_np[:,16]==None)[0],16] = 'Not Specified'   # convert None type users to Not Specified

for id in Pat_dic.keys():
    
    # extract team members
    users , user_counts = np.unique(AL_np[np.where(AL_np[:,2]==int(id))[0],16], return_counts = True)
    Pat_dic[id]['team_size'] = np.sum(user_counts>=AL_limit)
    Pat_dic[id]['team_members_id'] = users[np.where(user_counts>=AL_limit)[0]]
    
    # initialize
    int_id = int(id)
    temp_list = []
    temp_role_list= []
    print(id)
    # Extract team member previous experiences
    for user in Pat_dic[id]['team_members_id']:
        
        
        indx_all = np.where(AL_np[:,2]==int(id))[0] # find the index of the patient with encounter id: id
    
        # extract user prev experience
        indx = np.min(indx_all)
        te = np.where( (AL_np[:,16]==user) & (AL_np[:,-1]<= AL_np[indx,-1]) & (AL_np[:,2] != int_id) )[0]  #& (AL_np[:,-1]< AL_np[indx,-1]) & (AL_np[:,2] != int_id)
        n = np.unique(AL_np[te,2]).shape[0]
        temp_list.append(n)
        
        # extract user role
        temp_role_list.append( AL_np[np.where( (AL_np[:,2]==int(id)) & (AL_np[:,16]==user))[0],25][0])
    
                              
    Pat_dic[id]['user_prev_expr'] = np.array(temp_list)
    Pat_dic[id]['team_members_role'] = temp_role_list
    
    # report the team members of some of the cohort patients
    if i<=3:
        print(Pat_dic[id]['team_members_id'])
        print(Pat_dic[id]['user_prev_expr'])
        print(Pat_dic[id]['team_members_role'])
    
    i = i+1




In [5]:
# Create a table of cohort AL (v2: within 1 hour before and after tpa) with real times and Real time data into numpy

client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

query =  """
SELECT AL.*, DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real
  FROM `noshad.cohort_AL_60` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
  ORDER BY AL.rit_uid
"""
cursor.execute(query);

# into a numpy array
results = cursor.fetchall();
AL_np = np.array(results)

print(AL_np[0])

print('Real time data into numpy')



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated


KeyboardInterrupt: 

In [ ]:
# Run if I only want the treatment team information
# extract team members and size for each patient where the number of actions for each team member > AL_limit
AL_limit=3
i=0

AL_np[np.where(AL_np[:,16]==None)[0],16] = 'Not Specified'   # convert None type users to Not Specified

for id in Pat_dic.keys():
    
    # extract team members
    users , user_counts = np.unique(AL_np[np.where(AL_np[:,2]==int(id))[0],16], return_counts = True)
    Pat_dic[id]['team_size'] = np.sum(user_counts>=AL_limit)
    Pat_dic[id]['team_members_id'] = users[np.where(user_counts>=AL_limit)[0]]
    
    # report the team members of some of the cohort patients
    if i<=3:
        print(Pat_dic[id]['team_members_id'])
        #print(Pat_dic[id]['user_prev_expr'])
        #print(Pat_dic[id]['team_members_role'])
    
    i = i+1





In [ ]:
# New definition for the treatment team: 60 min before and after the tpa admin time
# override AL_np with the new access log selection
import time 

# extract team members and size for each patient where the number of actions for each team member > AL_limit

AL_limit=3
i=0

AL_np[np.where(AL_np[:,16]==None)[0],16] = 'Not Specified'   # convert None type users to Not Specified

for id in Pat_dic.keys():
    print(i, ' out of ' ,len(Pat_dic.keys()))
    
    # extract team members
    users , user_counts = np.unique(AL_np[np.where(AL_np[:,2]==int(id))[0],16], return_counts = True)
    Pat_dic[id]['team_size_60'] = np.sum(user_counts>=AL_limit)
    Pat_dic[id]['team_members_id_60'] = users[np.where(user_counts>=AL_limit)[0]]
    
    # initialize
    int_id = int(id)
    temp_list = []
    temp_role_list= []
    
    indx_all = np.where(AL_np[:,2]==int(id))[0] # find the index of the patient with encounter id: id
    
    # Extract team member previous experiences
    for user in Pat_dic[id]['team_members_id_60']:
        
    
        # extract user prev experience
        indx = np.min(indx_all)
        te = np.where( (AL_np[:,16]==user) & (AL_np[:,-1]<= AL_np[indx,-1]) & (AL_np[:,2] != int_id) )[0]  #& (AL_np[:,-1]< AL_np[indx,-1]) & (AL_np[:,2] != int_id)
        #print(user)
        #print(np.unique(AL_np[te,2]))
        n = np.unique(AL_np[te,2]).shape[0]
        temp_list.append(n)
        
        # extract user role
        temp_role_list.append( AL_np[np.where( (AL_np[:,2]==int(id)) & (AL_np[:,16]==user))[0],25][0])
    
                              
    Pat_dic[id]['user_prev_expr_60'] = np.array(temp_list)
    Pat_dic[id]['team_members_role_60'] = temp_role_list
    Pat_dic[id]['user_exp_score_60'] = np.sum(Pat_dic[id]['user_prev_expr_60'])
    
    # report the team members of some of the cohort patients
    if i<=3:
        #time.sleep(300)
        print(Pat_dic[id]['team_members_id_60'])
        print(Pat_dic[id]['user_prev_expr_60'])
        print(Pat_dic[id]['team_members_role_60'])
        print(Pat_dic[id]['user_exp_score_60'])
    
    i = i+1





In [ ]:
# report the patients with score less than 10
for id in Pat_dic.keys():
    if Pat_dic[id]['user_exp_score_60']<=5:
        print(Pat_dic[id]['team_members_id_60'])
        print(Pat_dic[id]['user_prev_expr_60'])
        print(Pat_dic[id]['team_members_role_60'])
        print(Pat_dic[id]['user_exp_score_60'])

In [ ]:
#save the results
X = np.array([Pat_dic[id]['user_exp_score_60'] for id in Pat_dic.keys()])
Time2tpa = np.array([Pat_dic[id]['t2tpa'] for id in Pat_dic.keys()])

np.save('features/user_exp_score',X)
np.save('features/t2tpa',Time2tpa)


In [ ]:
# plot feature user_exp_score
from sklearn.linear_model import LinearRegression as LR

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams.update({'font.size': 10})
plt.rcParams['axes.labelweight'] = 'bold'
import time



X = np.load('features/user_exp_score.npy')
Time2tpa = np.load('features/t2tpa.npy')

# remove outliers
D = np.where(X<500)[0]
print(D.shape)
X = X[D]
Time2tpa = Time2tpa[D]

plt.scatter(X,Time2tpa,color='r')

# Linear regression plot
X = np.reshape(X,(-1,1))
Time2tpa = np.reshape(Time2tpa,(-1,1))

print(X.shape)
print(Time2tpa.shape)

Time2tpa_regr = LR().fit(X, Time2tpa).predict(X)
plt.plot(X,Time2tpa_regr, color='blue', linewidth=3)

plt.xlabel('Individual Experience Score')
plt.ylabel('Time-to-TPA (min)')

timestr = time.strftime("%m_%d_%Y_%H%M%S")
plt.tight_layout()
plt.savefig('figures/'+timestr +'_user_score' +'.png', dpi=300)

plt.show()

In [ ]:
# repeat with ignoring low values:
# plot feature user_exp_score
from sklearn.linear_model import LinearRegression as LR
rcParams['font.family'] = 'serif'
rcParams.update({'font.size': 10})
plt.rcParams['axes.labelweight'] = 'bold'
import time



X = np.load('features/user_exp_score.npy')
Time2tpa = np.load('features/t2tpa.npy')

# remove outliers
D = np.where((X<500) & (40<X) )[0]
print(D.shape)
X = X[D]
Time2tpa = Time2tpa[D]

plt.scatter(X,Time2tpa,color='r')

# Linear regression plot
X = np.reshape(X,(-1,1))
Time2tpa = np.reshape(Time2tpa,(-1,1))

print(X.shape)
print(Time2tpa.shape)

Time2tpa_regr = LR().fit(X, Time2tpa).predict(X)
plt.plot(X,Time2tpa_regr, color='blue', linewidth=3)

plt.xlabel('Individual Experience Score')
plt.ylabel('Time-to-TPA (min)')


timestr = time.strftime("%m_%d_%Y_%H%M%S")
plt.tight_layout()
plt.savefig('figures/'+timestr +'_user_score' +'.png', dpi=300)

plt.show()

In [9]:
## Extract team members


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

query =  """
SELECT pat_enc_csn_id_coded, user_deid, count(*)
  FROM `noshad.cohort_AL_60` as AL
  GROUP BY pat_enc_csn_id_coded, user_deid

"""
print('Query Completed')
cursor.execute(query);

# into a numpy array
results = cursor.fetchall();
Users = np.array(results)

print(Users[0])

print('Into numpy')






/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
Query Completed
[131071934721 'SS0067345' 99]
Into numpy


In [10]:
for id in np.unique(Users[:,0]):
    Pat_dic[id]['team_members_id_60'] = Users[np.where(Users[:,0]==id)[0],1]
    Pat_dic[id]['user_num_action'] = Users[np.where(Users[:,0]==id)[0],2]
print(Pat_dic[id]['team_members_id_60'] )

KeyError: 131011501539

In [ ]:
## Joint team experience score

# Extract ordered tpa times:

client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')

query =  """
SELECT AL.pat_enc_csn_id_coded, min(DATETIME_SUB(AL.tpaAdminTime, INTERVAL TMP.JITTER DAY)) as tpa_time
  FROM `noshad.cohort_AL_60` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid = TMP.ANON_ID
  GROUP BY AL.pat_enc_csn_id_coded
  ORDER BY tpa_time

"""
print('Query Completed')
cursor.execute(query);

# into a numpy array
results = cursor.fetchall();
TPA_times_np = np.array(results)

print(TPA_times_np[0])

print('Real time data into numpy')





In [ ]:
print(TPA_times_np[0:10])

In [ ]:
Edge_Weight={}
Vertex_Weight = {}

for id in Pat_dic.keys():
    temp_sum_edge = 0
    for u1 in Pat_dic[id]['team_members_id_60']:
        
        temp_user_exp_list=[]
        
        #Count the vertex weights
        if u1 not in Vertex_Weight.keys():
                Vertex_Weight[u1] = 0
            else:
                temp_user_exp_list.append(Vertex_Weight[u1])
                Vertex_Weight[u1] += 1
        
        #Count the edge weights
        for u2 in Pat_dic[id]['team_members_id_60']:
            if (u1,u2) not in Edge_Weight.keys():
                Edge_Weight[(u1,u2)] = 0
            else:
                temp_sum_edge += Edge_Weight[(u1,u2)]
                Edge_Weight[(u1,u2)] += 1
                
    Pat_dic[id]['sum_edge_weight'] = temp_sum_edge
    if len(Pat_dic[id]['team_members_id_60']) == 0:
        #print(temp_sum_edge)
        Pat_dic[id]['avg_edge_weight'] = temp_sum_edge
    else:
        Pat_dic[id]['avg_edge_weight'] = temp_sum_edge / len(Pat_dic[id]['team_members_id_60'])
    
    # vertex weight (individual experience)
    Pat_dic[id]['user_prev_expr'] = temp_user_exp_list
    #Pat_dic[id]['user_prev_expr_60'] = np.array(temp_list)
    #Pat_dic[id]['team_members_role_60'] = temp_role_list
    Pat_dic[id]['user_exp_score_60'] = np.sum(np.array(temp_user_exp_list))
    
print(Pat_dic['131012861842']['sum_edge_weight'])
                    



In [ ]:
#save the results
X = np.array([Pat_dic[id]['sum_edge_weight'] for id in Pat_dic.keys()])
np.save('features/sum_user_joint_exp',X)

X = np.array([Pat_dic[id]['avg_edge_weight'] for id in Pat_dic.keys()])
np.save('features/avg_user_joint_exp',X)

X = np.array([len(Pat_dic[id]['team_members_id_60']) for id in Pat_dic.keys()])
np.save('features/team_size',X)

In [ ]:
# plot feature user_joint_exp_score
from sklearn.linear_model import LinearRegression as LR
rcParams['font.family'] = 'serif'
rcParams.update({'font.size': 10})
plt.rcParams['axes.labelweight'] = 'bold'
import time



X = np.load('features/sum_user_joint_exp.npy')
Time2tpa = np.load('features/t2tpa.npy')

# remove outliers
D = np.where((X<500) & (40<X) )[0]
print(D.shape)
X = X[D]
Time2tpa = Time2tpa[D]

plt.scatter(X,Time2tpa,color='r')

# Linear regression plot
X = np.reshape(X,(-1,1))
Time2tpa = np.reshape(Time2tpa,(-1,1))

print(X.shape)
print(Time2tpa.shape)

Time2tpa_regr = LR().fit(X, Time2tpa).predict(X)
plt.plot(X,Time2tpa_regr, color='blue', linewidth=3)

plt.xlabel('Joint Experience Score')
plt.ylabel('Time-to-TPA (min)')

timestr = time.strftime("%m_%d_%Y_%H%M%S")
plt.tight_layout()
plt.savefig('figures/'+timestr +'_user_joint_score' +'.png', dpi=300)

plt.show()

In [ ]:
# plot feature user_joint_exp_score
from sklearn.linear_model import LinearRegression as LR
rcParams['font.family'] = 'serif'
rcParams.update({'font.size': 10})
plt.rcParams['axes.labelweight'] = 'bold'
import time



X = np.load('features/avg_user_joint_exp.npy')
Time2tpa = np.load('features/t2tpa.npy')

# remove outliers
D = np.where((X<500) & (40<X) )[0]
print(D.shape)
X = X[D]
Time2tpa = Time2tpa[D]

plt.scatter(X,Time2tpa,color='r')

# Linear regression plot
X = np.reshape(X,(-1,1))
Time2tpa = np.reshape(Time2tpa,(-1,1))

print(X.shape)
print(Time2tpa.shape)

Time2tpa_regr = LR().fit(X, Time2tpa).predict(X)
plt.plot(X,Time2tpa_regr, color='blue', linewidth=3)

plt.xlabel('Joint Experience Score')
plt.ylabel('Time-to-TPA (min)')

timestr = time.strftime("%m_%d_%Y_%H%M%S")
plt.tight_layout()
plt.savefig('figures/'+timestr +'_user_joint_score' +'.png', dpi=300)

plt.show()

In [ ]:
# plot feature user_joint_exp_score
from sklearn.linear_model import LinearRegression as LR
rcParams['font.family'] = 'serif'
rcParams.update({'font.size': 10})
plt.rcParams['axes.labelweight'] = 'bold'
import time



X = np.load('features/team_size.npy')
Time2tpa = np.load('features/t2tpa.npy')

# remove outliers
D = np.where((X<500) & (40<X) )[0]
print(D.shape)
X = X[D]
Time2tpa = Time2tpa[D]

plt.scatter(X,Time2tpa,color='r')

# Linear regression plot
X = np.reshape(X,(-1,1))
Time2tpa = np.reshape(Time2tpa,(-1,1))

print(X.shape)
print(Time2tpa.shape)

Time2tpa_regr = LR().fit(X, Time2tpa).predict(X)
plt.plot(X,Time2tpa_regr, color='blue', linewidth=3)

plt.xlabel('Joint Experience Score')
plt.ylabel('Time-to-TPA (min)')

timestr = time.strftime("%m_%d_%Y_%H%M%S")
plt.tight_layout()
plt.savefig('figures/'+timestr +'_user_joint_score' +'.png', dpi=300)

plt.show()